# Proyecto: Cross Validation con Regresión Lineal

## Después de entrenar un algoritmo es importante asignarle una calificación (o nivel de confinaza)
## para saber con que precisión seran sus predicciones. Para esto usaremos la técnica Validación Cruzada
## o "Cross Validation" https://scikit-learn.org/stable/modules/cross_validation.html 

## A continuación veremos la estructura básica del proceso de validación cruzada. 

## Mostraremos un ejemplo con sólo 10 datos para poder visualizar el efecto de  la Validación Cruzada en los índices de los datos.

## Primero lo haremos manualmente constuyendeo el ciclo 'for' correspondiente a los diferentes
## conjuntos de entrenamiento y pruebas que se generan con `KFold.split()`
## Posteriormente lo haremos de forma automática usando las funciones: `cross_val_score` y `cross_val_predict`

In [12]:
# paquete que contiene el modelo de regresión lineal
from sklearn import linear_model, metrics

# importar metricas de error y precision:
from sklearn.metrics import mean_squared_error, r2_score

# función para separar los datos en Cross-Validation:
from sklearn.model_selection import KFold

from sklearn.model_selection import cross_val_score , cross_val_predict 

from sklearn.model_selection import train_test_split

import pandas as pd 

import numpy as np 

In [2]:
# Cargamos datos:
df = pd.read_csv('./Data_bases/moviesc10.csv')
df#.head()

,cast_total_facebook_likes,imdb_score
0,108016,5.4
1,303717,5.2
2,656730,7.2
3,137712,5.7
4,120797,6.0
5,140268,6.0
6,170118,6.4
7,261818,9.1
8,263584,5.6
9,283939,3.9


In [3]:
# Extraemos features y labels:
features = df['cast_total_facebook_likes'].values
labels = df['imdb_score'].values


# Redimensionamos los features como vectores columna (OBLIGATORIO PARA REGRESION LINEAL 1D):
features = features.reshape([ len(features) , 1 ])

In [4]:
# Separamos Datos para entrenamiento y Pruebas:
X_train, X_test, y_train, y_test = train_test_split( features, labels, test_size=0.2, random_state=0)

## Declaramos el modelo y lo entrenamos:

In [5]:
# Declaramos el modelo de regresion lineal:
regr = linear_model.LinearRegression()

In [6]:
# Entrenamos el modelo con los datos de entrenamiento: 
regr.fit(X_train , y_train)

LinearRegression()

## Evaluamos el aprendizaje del modelo (usando la función `score`):

In [7]:
# Evaluación con datos de entrenamiento:
acc_train = regr.score(X_train , y_train)

# Evaluación con datos de prueba:
acc_test = regr.score(X_test , y_test)

print('Accuracy con datos de entrenamiento:', acc_train)
print('Accuracy con datos de prueba:', acc_test)

Accuracy con datos de entrenamiento: 0.0001062042440163058
Accuracy con datos de prueba: -0.47041399147274654


## Evaluamos el aprendizaje del modelo mediante Cross Validation:

In [8]:
# Definimos el separador K-Fold para hacer Cross-Validation:

n_subsets = 5 # numero de particiones para K-fold

kf = KFold(n_splits= n_subsets )#, shuffle=True, random_state=2)

In [9]:
# A continuación mostramos la separación de indices de entrenamiento y pruebas 
# que se usan en el proceso de Validación Cruzada mediante K-fold:

k=1
for id_train, id_test in kf.split(df):
    print('\nFold',k)
    print('Indices_train:',id_train)
    print('Indices_test' , id_test)
    k = k+1


Fold 1
Indices_train: [2 3 4 5 6 7 8 9]
Indices_test [0 1]

Fold 2
Indices_train: [0 1 4 5 6 7 8 9]
Indices_test [2 3]

Fold 3
Indices_train: [0 1 2 3 6 7 8 9]
Indices_test [4 5]

Fold 4
Indices_train: [0 1 2 3 4 5 8 9]
Indices_test [6 7]

Fold 5
Indices_train: [0 1 2 3 4 5 6 7]
Indices_test [8 9]


# Proceso de Validación Cruzada usando un "ciclo for":

In [10]:
# Construimos el ciclo en donde se realiza la Cross-Validation:

loss = []
acc = []

k=1
for id_train, id_test in kf.split(df):
    
    # ***************** Entrenamiento *****************
    # numero de datos de entrenamiento:
    nk_train = len(id_train)

    # Extraemos features de entrenamiento:
    xk_train = features[id_train]

    # Extraemos labels de entrenamiento:
    yk_train = labels[id_train]    

    # Entrenamos:
    regr.fit(xk_train, yk_train)

    # ***************** Pruebas *****************
    # numero de datos de prueba:
    nk_test = len(id_test)

    # Extraemos features de prueba:
    xk_test = features[id_test]

    # Predecimos con datos de prueba:
    yk_test_pred = regr.predict(xk_test)    
    
    # Extraemos labels de pruebas:
    yk_test = labels[id_test]

    # ***************** Metricas *****************
    # Error:
    loss_k = mean_squared_error(yk_test, yk_test_pred)
    
    # Precisión:
    accuracy_k = r2_score(yk_test, yk_test_pred)
    # El accuracy también se puede obtener con: 'regr.score(xk_test, yk_test)'

    # Agregamos valores a las lista de loss y accuracy:
    loss.append( loss_k )
    acc.append( accuracy_k )

    print('\nFold',k)
    print('y_test_pred=',yk_test_pred)
    print('loss',loss_k)
    print('accuracy', accuracy_k)

    k = k+1


Fold 1
y_test_pred= [5.93627307 6.33906431]
loss 0.7925281551372426
accuracy -78.25281551372412

Fold 2
y_test_pred= [5.47278461 6.0229502 ]
loss 1.5437849205920788
accuracy -1.744506525497029

Fold 3
y_test_pred= [5.70473911 5.75044535]
loss 0.07472825672768767
accuracy 0.0

Fold 4
y_test_pred= [5.45074024 5.64626371]
loss 6.414694226270729
accuracy -2.5197224835504706

Fold 5
y_test_pred= [6.44256213 6.49507778]
loss 3.722169801942026
accuracy -4.151792113414571


In [13]:
# Accuracy y Loss:
loss = np.mean(loss)
acc = np.mean(acc)

print("Error promedio ",loss)

print("Precisión promedio ",acc)


Error promedio  2.509581072133953
Precisión promedio  -17.333767327237236


# Proceso de Validación Cruzada usando las funciones: `cross_val_score` y `cross_val_predict`


In [14]:
# Obtenemos la precisión en cada subconjunto de entrenamiento y pruebas del Cross Validation:
acc_CV = cross_val_score( regr , features, labels , cv=kf )
print(acc_CV)

[-78.25281551  -1.74450653   0.          -2.51972248  -4.15179211]


In [15]:
# Predicciones de labels correspondientes a los features de prueba en cada 
# conjunto de cross-validation
pred_CV = cross_val_predict(regr , features, labels , cv=kf)

print(pred_CV)

[5.93627307 6.33906431 5.47278461 6.0229502  5.70473911 5.75044535
 5.45074024 5.64626371 6.44256213 6.49507778]


In [17]:
# Accuracy y Loss:
acc_Real = np.mean(acc_CV)

print("Precisión promedio ",acc_Real)

Precisión promedio  -17.333767327237236
